## Implementing CNN using KERAS

In [8]:
# Importing libraries

from __future__ import print_function
import numpy as np
import keras
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras import backend as K

print("All libs imported.")

All libs imported.


In [9]:
# Loading the notMNIST dataset

pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


##### Reshaping dataset for CNN with input data as a 3D object and label as float 1-Hot Encoding 

In [10]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


### CNN  using Keras Sequential Model 1

In [4]:
# MODEL: conv -> conv -> maxpool -> NN classifier with 1 hidden layer, dropouts

input_shape = [image_size, image_size, num_channels]
batch_size = 16
epochs = 12

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_dataset, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(valid_dataset, valid_labels))

score = model.evaluate(test_dataset, test_labels, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 200000 samples, validate on 10000 samples
Epoch 1/12
200000/200000 [==============================] - 1241s 6ms/step - loss: 0.4818 - acc: 0.8608 - val_loss: 0.3609 - val_acc: 0.8918
Epoch 2/12
200000/200000 [==============================] - 1300s 7ms/step - loss: 0.4311 - acc: 0.8768 - val_loss: 0.3542 - val_acc: 0.8964
Epoch 3/12
200000/200000 [==============================] - 1228s 6ms/step - loss: 0.4229 - acc: 0.8785 - val_loss: 0.3526 - val_acc: 0.8953
Epoch 4/12
200000/200000 [==============================] - 1146s 6ms/step - loss: 0.4222 - acc: 0.8797 - val_loss: 0.3511 - val_acc: 0.8968
Epoch 5/12
200000/200000 [==============================] - 1135s 6ms/step - loss: 0.4228 - acc: 0.8805 - val_loss: 0.3450 - val_acc: 0.8968
Epoch 6/12
200000/200000 [==============================] - 1159s 6ms/step - loss: 0.4234 - acc: 0.8809 - val_loss: 0.3501 - val_acc: 0.8983
Epoch 7/12
200000/200000 [==============================] - 1235s 6ms/step - loss: 0.4261 - acc: 0.8812

### CNN using Keras Sequential Model 2

In [1]:
# MODEL: conv -> maxpool -> conv -> maxpool -> NN classifier with 1 hidden layer, dropouts

input_shape = [image_size, image_size, num_channels]
batch_size = 200
epochs = 10

model = Sequential()

model.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding ='same'))
model.add(Dropout(0.20))
model.add(Conv2D(16, (3, 3), padding = 'same', activation ='relu'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding ='same'))
model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(num_labels, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_dataset, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(valid_dataset, valid_labels))

score = model.evaluate(test_dataset, test_labels, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'image_size' is not defined

### CNN  using Keras Sequential Model 3 (LeNet-ish)

In [12]:
# MODEL: conv -> maxpool -> conv -> maxpool -> NN classifier with 2 hidden layers, dropouts

input_shape = [image_size, image_size, num_channels]
batch_size = 200
epochs = 10

model = Sequential()

model.add(Conv2D(6, (5, 5), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding ='same'))
model.add(Dropout(0.20))
model.add(Conv2D(16, (5, 5), padding = 'same', activation ='relu'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding ='same'))
model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(84, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_labels, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(train_dataset, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(valid_dataset, valid_labels))

score = model.evaluate(test_dataset, test_labels, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 200000 samples, validate on 10000 samples
Epoch 1/10
200000/200000 [==============================] - 400s 2ms/step - loss: 0.5720 - acc: 0.8258 - val_loss: 0.3634 - val_acc: 0.8909
Epoch 2/10
200000/200000 [==============================] - 390s 2ms/step - loss: 0.4081 - acc: 0.8761 - val_loss: 0.3236 - val_acc: 0.9008
Epoch 3/10
200000/200000 [==============================] - 383s 2ms/step - loss: 0.3754 - acc: 0.8853 - val_loss: 0.3040 - val_acc: 0.9098
Epoch 4/10
200000/200000 [==============================] - 382s 2ms/step - loss: 0.3575 - acc: 0.8909 - val_loss: 0.2930 - val_acc: 0.9131
Epoch 5/10
200000/200000 [==============================] - 381s 2ms/step - loss: 0.3458 - acc: 0.8938 - val_loss: 0.2852 - val_acc: 0.9141
Epoch 6/10
200000/200000 [==============================] - 381s 2ms/step - loss: 0.3356 - acc: 0.8976 - val_loss: 0.2822 - val_acc: 0.9160
Epoch 7/10
200000/200000 [==============================] - 396s 2ms/step - loss: 0.3286 - acc: 0.8992 - val_